In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import json
import seaborn as sns
import re

%matplotlib inline 

In [2]:
member_df = pd.read_csv('../data/NSS_GreatCare_Members.csv')
member_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61519 entries, 0 to 61518
Data columns (total 63 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   Patient ID                                                       61519 non-null  object 
 1   Primary Plan ID                                                  61519 non-null  object 
 2   Primary Payor ID                                                 61519 non-null  object 
 3   Is Patient Alive                                                 61519 non-null  int64  
 4   Patient Age Integer                                              61519 non-null  int64  
 5   Patient Date of Birth                                            61519 non-null  object 
 6   Patient ZIP Code 5-Digit                                         61105 non-null  float64
 7   Count Primary Care Visits in Last 36 Mon

In [3]:
inpatient_admin = pd.read_csv('../data/NSS_Hospital_Inpatient_Admissions.csv')
inpatient_admin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 23 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Encounter ID                                         6213 non-null   object 
 1   Patient ID                                           6213 non-null   object 
 2   Encounter Primary Payor ID                           6213 non-null   object 
 3   Encounter Primary Plan ID                            6213 non-null   object 
 4   Facility Provider ID                                 6213 non-null   object 
 5   Attending Provider ID                                5884 non-null   object 
 6   Attending Provider NPI                               5884 non-null   float64
 7   Facility Provider NPI                                6213 non-null   object 
 8   Encounter Start Date                                 6213 non-null  

In [4]:
# Converting some data types, specifically the dates

inpatient_admin[['Encounter Start Date','Encounter End Date']] = inpatient_admin[['Encounter Start Date','Encounter End Date']].apply(pd.to_datetime)
inpatient_admin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 23 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   Encounter ID                                         6213 non-null   object        
 1   Patient ID                                           6213 non-null   object        
 2   Encounter Primary Payor ID                           6213 non-null   object        
 3   Encounter Primary Plan ID                            6213 non-null   object        
 4   Facility Provider ID                                 6213 non-null   object        
 5   Attending Provider ID                                5884 non-null   object        
 6   Attending Provider NPI                               5884 non-null   float64       
 7   Facility Provider NPI                                6213 non-null   object        
 8 

In [5]:
pcp_visit = pd.read_csv('../data/NSS_PrimaryCare_Office_Visits.csv',low_memory = False)
pcp_visit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231373 entries, 0 to 231372
Data columns (total 18 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   Encounter ID                                   231373 non-null  object 
 1   Patient ID                                     231373 non-null  object 
 2   Service Provider ID                            230599 non-null  object 
 3   Attending Provider ID                          39644 non-null   object 
 4   Provider Group Provider ID                     227644 non-null  object 
 5   Facility Provider ID                           39756 non-null   object 
 6   Encounter Primary Payor ID                     231373 non-null  object 
 7   Encounter Primary Plan ID                      231373 non-null  object 
 8   Attending Provider NPI                         39644 non-null   float64
 9   Facility Provider NPI                

In [6]:
pcp_visit['Encounter Date'] = pd.to_datetime(pcp_visit['Encounter Date'])
pcp_visit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231373 entries, 0 to 231372
Data columns (total 18 columns):
 #   Column                                         Non-Null Count   Dtype         
---  ------                                         --------------   -----         
 0   Encounter ID                                   231373 non-null  object        
 1   Patient ID                                     231373 non-null  object        
 2   Service Provider ID                            230599 non-null  object        
 3   Attending Provider ID                          39644 non-null   object        
 4   Provider Group Provider ID                     227644 non-null  object        
 5   Facility Provider ID                           39756 non-null   object        
 6   Encounter Primary Payor ID                     231373 non-null  object        
 7   Encounter Primary Plan ID                      231373 non-null  object        
 8   Attending Provider NPI                      

# Q2. What percent of patients have a PCP visit within a week of discharge?

In [7]:
# Going to start on question 2 cause I think that's easier

# Gonna pick the columns I need from 'Inpatient' and 'PCP' and then join them

x = inpatient_admin[['Encounter ID','Patient ID', 'Encounter Start Date', 'Encounter End Date']]
y = pcp_visit[['Encounter ID','Patient ID', 'Encounter Date']]

print(x)
print(y)

                            Encounter ID Patient ID Encounter Start Date  \
0     HIA|DS1|10033|DS1|NPI|1306890389|1  DS1|10033           2013-01-12   
1     HIA|DS1|10043|DS1|NPI|1902803315|1  DS1|10043           2014-01-05   
2     HIA|DS1|10073|DS1|NPI|1902803315|1  DS1|10073           2012-06-13   
3     HIA|DS1|10101|DS1|NPI|1427055839|1  DS1|10101           2012-12-06   
4     HIA|DS1|10118|DS1|NPI|1992818256|1  DS1|10118           2015-03-11   
...                                  ...        ...                  ...   
6208   HIA|DS1|9979|DS1|NPI|1689772592|1   DS1|9979           2015-02-06   
6209   HIA|DS1|9981|DS1|NPI|1427055839|1   DS1|9981           2014-08-07   
6210   HIA|DS1|9994|DS1|NPI|1689608150|1   DS1|9994           2015-11-21   
6211   HIA|DS1|9994|DS1|NPI|1689608150|2   DS1|9994           2015-11-24   
6212   HIA|DS1|9996|DS1|NPI|1033298617|1   DS1|9996           2010-03-28   

     Encounter End Date  
0            2013-01-14  
1            2014-01-12  
2        

In [8]:
# Merge. There will be duplicates because of the mismatch dates between PCP visits and Inpatient Visits.

inpatient_pcp_df = x.merge(y, how = 'left', on = 'Patient ID')
inpatient_pcp_df

,Encounter ID_x,Patient ID,Encounter Start Date,Encounter End Date,Encounter ID_y,Encounter Date
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1228384|1043427248|1|SHIFTED,2015-12-08
1,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1326684|1043427248|1|SHIFTED,2012-06-18
2,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|155317|1043427248|1|SHIFTED,2014-06-10
3,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|439093|1043427248|1|SHIFTED,2014-09-28
4,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|711310|1043427248|1|SHIFTED,2014-03-16
...,...,...,...,...,...,...
44093,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|1238517|1407058787|2,2015-12-05
44094,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|302749|1407058787|2,2015-12-13
44095,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|781849|1205884392|1,2014-05-18
44096,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|93182|1043427248|1,2013-07-20


In [9]:
# Time to find if they visited the pcp after hospital

inpatient_pcp_df['7_Days'] = inpatient_pcp_df['Encounter Date'] - inpatient_pcp_df['Encounter End Date']
inpatient_pcp_df

,Encounter ID_x,Patient ID,Encounter Start Date,Encounter End Date,Encounter ID_y,Encounter Date,7_Days
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1228384|1043427248|1|SHIFTED,2015-12-08,1058 days
1,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1326684|1043427248|1|SHIFTED,2012-06-18,-210 days
2,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|155317|1043427248|1|SHIFTED,2014-06-10,512 days
3,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|439093|1043427248|1|SHIFTED,2014-09-28,622 days
4,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|711310|1043427248|1|SHIFTED,2014-03-16,426 days
...,...,...,...,...,...,...,...
44093,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|1238517|1407058787|2,2015-12-05,8 days
44094,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|302749|1407058787|2,2015-12-13,16 days
44095,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|781849|1205884392|1,2014-05-18,-558 days
44096,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|93182|1043427248|1,2013-07-20,-860 days


In [10]:
# Looping through the time difference column to find any that are within 7days. 

t = []

for i in inpatient_pcp_df['7_Days']:
    if (i <= dt.timedelta(days=7)) and (i >= dt.timedelta(days=0)) :
        t.append('yes')
    else:
        t.append('no')

In [11]:
# Adding my column 

inpatient_pcp_df['within_7'] = t

In [12]:
inpatient_pcp_df

,Encounter ID_x,Patient ID,Encounter Start Date,Encounter End Date,Encounter ID_y,Encounter Date,7_Days,within_7
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1228384|1043427248|1|SHIFTED,2015-12-08,1058 days,no
1,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1326684|1043427248|1|SHIFTED,2012-06-18,-210 days,no
2,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|155317|1043427248|1|SHIFTED,2014-06-10,512 days,no
3,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|439093|1043427248|1|SHIFTED,2014-09-28,622 days,no
4,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|711310|1043427248|1|SHIFTED,2014-03-16,426 days,no
...,...,...,...,...,...,...,...,...
44093,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|1238517|1407058787|2,2015-12-05,8 days,no
44094,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|302749|1407058787|2,2015-12-13,16 days,no
44095,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|781849|1205884392|1,2014-05-18,-558 days,no
44096,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|93182|1043427248|1,2013-07-20,-860 days,no


In [13]:
# There are 2362 instances in which a patient visited their PCP within 7 days of an Inpatient visit. 

inpatient_pcp_df['within_7'].value_counts()

no     41723
yes     2375
Name: within_7, dtype: int64

# So there are duplicates

There seems to be duplicate entries because of multiple doctor visits. Example: Patient is discharged on 1/10/2021 and they visit their PCP on 1/12, 1/14, and 1/16. \
This creates 3 duplicate entries for the 1 hospital visit \
I could elminate this by counting each 'Encounter ID_x' as 1 instance \
\
I will clean up down below

In [14]:
# To finish Q2, I will filter out all of the 'no' and be left with only 'yes'. I can merge that back to the original 'Inpatient' DF to find a %

y = inpatient_pcp_df.loc[inpatient_pcp_df['within_7'] == 'yes']

# Using this filtered df, I will remove duplicates mentioned in my Markdown above. This way, each hospital encounter will only count 1 time if they visit a doc after.
y = y.drop_duplicates(subset='Encounter ID_x', keep='first')
y

,Encounter ID_x,Patient ID,Encounter Start Date,Encounter End Date,Encounter ID_y,Encounter Date,7_Days,within_7
17,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,2014-01-05,2014-01-12,COV|PBSL|DS1|10043|P|538933|1043427248|1|SHIFTED,2014-01-12,0 days,yes
28,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,2012-06-13,2012-06-21,COV|PBSL|DS1|10073|P|301709|1043427248|1|SHIFTED,2012-06-27,6 days,yes
50,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,2012-12-06,2012-12-09,COV|PCSL|DS1|10101|P|1189971|1043427248|1,2012-12-12,3 days,yes
231,HIA|DS1|10303|DS1|NPI|1710918545|1,DS1|10303,2012-06-21,2012-06-24,COV|PBSL|DS1|10303|P|10896|1912987553|1,2012-06-30,6 days,yes
247,HIA|DS1|10305|DS1|NPI|1184722779|1,DS1|10305,2014-05-10,2014-05-12,COV|PBSL|DS1|10305|P|823298|1184722779|1,2014-05-13,1 days,yes
...,...,...,...,...,...,...,...,...
44039,HIA|DS1|9943|DS1|NPI|1184722779|3,DS1|9943,2014-10-28,2014-10-30,COV|PCSL|DS1|9943|P|1315984|1184722779|1,2014-11-03,4 days,yes
44051,HIA|DS1|9945|DS1|NPI|1902803315|1,DS1|9945,2013-02-09,2013-02-11,COV|PCSL|DS1|9945|P|840374|1043427248|1,2013-02-14,3 days,yes
44067,HIA|DS1|9978|DS1|NPI|1427055839|1,DS1|9978,2015-09-01,2015-09-05,COV|PCSL|DS1|9978|P|696226|1043427248|1,2015-09-11,6 days,yes
44071,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,2015-02-06,2015-02-11,COV|PCSL|DS1|9979|P|281365|1477666352|1,2015-02-15,4 days,yes


In [15]:
# Need to clean up my columns to get ready to merge

w = y[['Encounter ID_x','Encounter Date','within_7']]
w

,Encounter ID_x,Encounter Date,within_7
17,HIA|DS1|10043|DS1|NPI|1902803315|1,2014-01-12,yes
28,HIA|DS1|10073|DS1|NPI|1902803315|1,2012-06-27,yes
50,HIA|DS1|10101|DS1|NPI|1427055839|1,2012-12-12,yes
231,HIA|DS1|10303|DS1|NPI|1710918545|1,2012-06-30,yes
247,HIA|DS1|10305|DS1|NPI|1184722779|1,2014-05-13,yes
...,...,...,...
44039,HIA|DS1|9943|DS1|NPI|1184722779|3,2014-11-03,yes
44051,HIA|DS1|9945|DS1|NPI|1902803315|1,2013-02-14,yes
44067,HIA|DS1|9978|DS1|NPI|1427055839|1,2015-09-11,yes
44071,HIA|DS1|9979|DS1|NPI|1689772592|1,2015-02-15,yes


In [16]:
# Merge

doc_7_day = inpatient_admin.merge(w, how = 'left', left_on = 'Encounter ID', right_on = 'Encounter ID_x')
doc_7_day

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,Encounter ID_x,Encounter Date,within_7
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,NaN,NaT,NaN
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,HIA|DS1|10043|DS1|NPI|1902803315|1,2014-01-12,yes
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,HIA|DS1|10073|DS1|NPI|1902803315|1,2012-06-27,yes
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,HIA|DS1|10101|DS1|NPI|1427055839|1,2012-12-12,yes
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,...,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,HIA|DS1|9979|DS1|NPI|1689772592|1,2015-02-15,yes
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,...,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,HIA|DS1|9981|DS1|NPI|1427055839|1,2014-08-11,yes
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,...,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,NaN,NaT,NaN
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,...,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,NaN,NaT,NaN


In [17]:
# Little bit of cleaning up. Dropping a column and filling in 'NaN' with 'no'.

doc_7_day = doc_7_day.drop(['Encounter ID_x'], axis = 1)
doc_7_day['within_7'] = doc_7_day['within_7'].fillna('no')
doc_7_day

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,Encounter Date,within_7
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,NaT,no
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,2014-01-12,yes
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,2012-06-27,yes
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,2012-12-12,yes
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0,NaT,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,...,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,2015-02-15,yes
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,...,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,2014-08-11,yes
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,...,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,NaT,no
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,...,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,NaT,no


In [18]:
# 1864(doc visits) out of the total 6213(hospital visits). 30.00% went to their PCP within 7 days after the hospital discharge.

print(doc_7_day['within_7'].value_counts())
print(1864/6213*100)

no     4349
yes    1864
Name: within_7, dtype: int64
30.001609528408174


# Q1. What is our 30-day all cause readmission rate across hospitals our members are admitted to?
\
Code from Debbie:\
thirtyadm['next_adm']=thirtyadm.sort_values(by=['start_date'],ascending=True).groupby(['patient_id'])['start_date'].shift(-1)


In [19]:
# Working in a temp variable so I don't mess up my original df.
q = inpatient_admin
q

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,2,1,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,...,2,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,...,1,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,...,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,...,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0


In [20]:
# Code from Debbie works great. It adds a column and puts the next date they visited the hospital. I can reuse my work from the doctor calculations to find 30 day readmission.

q['next_admin'] =  inpatient_admin.sort_values(by=['Encounter Start Date'],ascending=True).groupby(['Patient ID'])['Encounter Start Date'].shift(-1)
q

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,next_admin
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,NaT
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,NaT
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,NaT
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,NaT
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,1,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,...,6,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,NaT
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,...,1,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,NaT
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,...,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,2015-11-24
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,...,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,NaT


In [21]:
# Lets add a column to see the time diff between end of encounter to next admission

q['time_diff'] = q['next_admin'] - q['Encounter End Date']
q

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,next_admin,time_diff
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,NaT,NaT
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,NaT,NaT
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,NaT,NaT
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,NaT,NaT
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,2015-02-11,...,21.0,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,NaT,NaT
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,2014-08-10,...,21.0,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,NaT,NaT
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,...,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,2015-11-24,1 days
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,...,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,NaT,NaT


In [22]:
b = []

for i in q['time_diff']:
    if (i <= dt.timedelta(days=30)) and (i >= dt.timedelta(days = 0)):
        b.append('yes')
    else: 
        b.append('no')

In [23]:
# 696 instances, the patient was readmitted within 30 days. 11.20% readmission

q['within_30'] = b
print(q['within_30'].value_counts())
print(696/6213 * 100)

no     5517
yes     696
Name: within_30, dtype: int64
11.202317720907775


# Q3. Do either vary by individual or group PCP's?
\
Not quite sure how to answer this question. I could group the hospital visits by the individual PCP or group PCPs to find out thier percentages. 

In [29]:
y = doc_7_day[doc_7_day['within_7']=='yes']
n = doc_7_day[doc_7_day['within_7']=='no']

In [35]:
g = y['Encounter ID'].count()/doc_7_day.count()
g

1864